In [19]:
import sys
sys.path.append('.')
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import torch
%matplotlib inline

## Train Data

In [21]:
train_df = pd.read_csv('inputs/TRAIN.txt', sep='\t')
print('{0:,} registros a las {1}'.format(len(train_df), datetime.now()))
train_df.head()

60,797 registros a las 2020-11-09 08:27:17.258395


,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
0,50833,2253302,0,S,2,1,192034,58.16,2006,40,34.52,46.58
1,67770,700754,0,S,2,1,3891897,30.95,2004,20,30.99,16.98
2,71253,528795,0,S,2,1,245070,49.99,2010,32,14.01,30.98
3,31295,901250,0,D,4,1,3480889,390.00,2004,21,0.00,73.13
4,43339,578450,0,S,5,1,1422247,51.95,1990,35,4.99,35.99


In [22]:
train_df.sort_values(['ORDEN', 'SKU'], ascending=[True, True], inplace=True)
print('{0:,} registros a las {1}'.format(len(train_df), datetime.now()))
train_df.head()

60,797 registros a las 2020-11-09 08:27:20.314136


,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
58481,2,1737127,0,D,15,1,682743,44.99,2015,8,28.97,31.84
42622,3,3255963,0,D,7,1,1016014,24.81,2005,39,0.00,15.54
19535,4,612701,0,D,0,0,340464,46.00,2013,34,30.19,27.97
13680,6,115883,1,D,4,1,334011,100.00,2006,20,133.93,83.15
11324,7,863939,1,D,2,1,1287938,121.95,2010,28,4.00,23.99


In [23]:
train_df.dtypes

ORDEN                  int64
SKU                    int64
VENTA_ALTA             int64
MARKETING             object
ACTUALIZACIONES        int64
ACTUALIZADO            int64
VENTAS_ACUMULADAS      int64
PRICE                float64
LANZAMIENTO            int64
CANTIDAD               int64
PRECIO_MIN           float64
SD_PRECIO            float64
dtype: object

In [24]:
len(train_df['SKU'].unique())

60797

Articulos por orden??

In [5]:
grouped_1_df = train_df.groupby('ORDEN').agg({'SKU':'nunique'}).reset_index()
grouped_1_df

,ORDEN,SKU
0,2,1
1,3,1
2,4,1
3,6,1
4,7,1
...,...,...
60792,85101,1
60793,85102,1
60794,85103,1
60795,85104,1


In [5]:
grouped_1_df['SKU'].value_counts()

1    60797
Name: SKU, dtype: int64

In [6]:
train_df.isna().sum()

ORDEN                0
SKU                  0
VENTA_ALTA           0
MARKETING            0
ACTUALIZACIONES      0
ACTUALIZADO          0
VENTAS_ACUMULADAS    0
PRICE                0
LANZAMIENTO          0
CANTIDAD             0
PRECIO_MIN           0
SD_PRECIO            0
dtype: int64

In [7]:
train_df.describe()

,ORDEN,SKU,VENTA_ALTA,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
count,60797.000000,6.079700e+04,60797.000000,60797.000000,60797.000000,6.079700e+04,60797.000000,60797.000000,60797.000000,60797.000000,60797.000000
mean,41643.421633,6.529548e+05,0.170699,4.144497,0.788411,1.219195e+06,98.963463,2005.984785,43.821965,56.680905,47.288439
std,25057.005884,6.875584e+05,0.376249,3.945417,0.408439,1.539134e+06,79.316429,6.107125,36.645181,106.498309,132.424575
min,2.000000,5.000100e+04,0.000000,0.000000,0.000000,1.250000e+02,0.000000,1935.000000,0.000000,0.000000,0.000000
25%,19462.000000,1.701200e+05,0.000000,2.000000,1.000000,2.431360e+05,49.950000,2003.000000,22.000000,20.400000,18.760000
50%,41498.000000,5.409390e+05,0.000000,3.000000,1.000000,7.117010e+05,78.950000,2007.000000,34.000000,43.990000,36.060000
75%,63563.000000,7.589470e+05,0.000000,6.000000,1.000000,1.550652e+06,127.950000,2010.000000,53.000000,79.290000,56.940000
max,85105.000000,3.959831e+06,1.000000,99.000000,1.000000,1.666966e+07,3986.310000,2017.000000,1523.000000,14140.210000,19138.790000


In [17]:
len(train_df[train_df['PRICE'] == .0])

1126

In [18]:
train_df[train_df['PRICE'] == .0]

,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
52190,16,3016759,0,D,2,1,4323581,0.0,1994,26,0.00,61.85
54073,27,2387463,1,D,6,1,270193,0.0,1979,22,141.98,76.22
15613,56,1447623,0,D,5,1,624574,0.0,2008,20,96.97,48.93
2831,83,705864,1,D,6,1,3364314,0.0,2012,22,158.81,30.00
49319,85,716418,1,D,12,1,3476445,0.0,2008,33,120.01,49.49
...,...,...,...,...,...,...,...,...,...,...,...,...
50310,84046,3101430,0,S,2,1,5314702,0.0,2005,11,48.98,0.00
59590,84526,881700,0,S,0,0,496908,0.0,2013,13,9.97,35.48
46829,84713,1439596,1,S,4,1,351841,0.0,2009,31,42.30,82.37
5666,84715,109645,0,S,2,1,1334108,0.0,2006,23,8.49,23.99


In [25]:
train_df[train_df['PRECIO_MIN'] == .0]

,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
42622,3,3255963,0,D,7,1,1016014,24.81,2005,39,0.0,15.54
11044,13,1597828,0,D,9,1,161348,36.95,2008,21,0.0,27.98
52190,16,3016759,0,D,2,1,4323581,0.00,1994,26,0.0,61.85
48500,57,1449431,0,D,2,1,1088497,130.75,2007,18,0.0,63.96
17368,63,184091,0,D,1,0,808413,140.95,2005,31,0.0,39.72
...,...,...,...,...,...,...,...,...,...,...,...,...
12168,85030,145548,0,S,9,1,870280,106.20,2006,28,0.0,0.00
46682,85032,719102,0,S,2,1,405563,55.00,1998,15,0.0,40.57
45204,85062,2067669,0,S,0,0,1008268,9.99,2008,12,0.0,9.99
35145,85067,661791,0,S,1,0,1498355,40.95,2010,14,0.0,41.03


In [26]:
train_df[train_df['SD_PRECIO'] == .0]

,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
20000,144,847859,0,D,3,1,6814589,0.00,1998,32,49.15,0.0
51760,358,613288,0,D,15,1,1701823,0.00,2012,9,130.99,0.0
45806,415,699076,0,D,3,1,1246086,96.75,2008,16,0.00,0.0
46059,419,112149,0,D,5,1,568370,110.75,2002,38,0.00,0.0
15741,579,111352,0,D,4,1,2283633,0.00,2009,11,170.48,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
53023,84879,55411,0,S,2,1,227223,106.20,2002,17,0.00,0.0
9991,84898,635953,0,S,2,1,135909,50.99,2011,26,0.00,0.0
19095,84950,127065,0,S,2,1,165328,49.95,2006,20,0.00,0.0
52819,84986,117386,0,S,8,1,352037,54.00,2008,30,0.00,0.0


In [28]:
len(train_df[(train_df['PRECIO_MIN'] == .0) & (train_df['SD_PRECIO'] == .0) & (train_df['PRICE'] == .0)])

31

In [9]:
train_df['ORDEN'].value_counts().unique()

array([1])

In [10]:
train_df['MARKETING'].value_counts()

S    32676
D    28121
Name: MARKETING, dtype: int64

In [11]:
train_df['ACTUALIZADO'].value_counts()

1    47933
0    12864
Name: ACTUALIZADO, dtype: int64

In [12]:
train_df['LANZAMIENTO'].value_counts()

2010    4981
2009    4752
2008    4735
2007    4661
2011    4509
        ... 
1959       1
1964       1
1954       1
2017       1
1953       1
Name: LANZAMIENTO, Length: 67, dtype: int64

## Validation Data

In [13]:
validation_df = pd.read_csv('TEST.txt', sep='\t')
print('{0:,} registros a las {1}'.format(len(validation_df), datetime.now()))
validation_df.head()

15,199 registros a las 2020-11-08 21:47:19.145514


,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
0,9,484059,NaN,D,13,1,2314801,95.95,2010,33,90.77,46.49
1,20,115159,NaN,D,1,0,909831,126.20,2001,33,38.92,44.46
2,24,178701,NaN,D,0,0,1814079,399.95,2012,34,71.79,91.90
3,28,2398402,NaN,D,2,1,156638,98.95,1988,67,142.71,14.02
4,39,576667,NaN,D,3,1,823824,123.95,2008,33,30.96,11.99


In [14]:
validation_df.isna().sum()

ORDEN                    0
SKU                      0
VENTA_ALTA           15199
MARKETING                0
ACTUALIZACIONES          0
ACTUALIZADO              0
VENTAS_ACUMULADAS        0
PRICE                    0
LANZAMIENTO              0
CANTIDAD                 0
PRECIO_MIN               0
SD_PRECIO                0
dtype: int64

In [16]:
np.log([0,1,2,3,4,5,10,11,100,1000])

<ipython-input-16-f172adf34963>:1: RuntimeWarning: divide by zero encountered in log
  np.log([0,1,2,3,4,5,10,11,100,1000])


array([      -inf, 0.        , 0.69314718, 1.09861229, 1.38629436,
       1.60943791, 2.30258509, 2.39789527, 4.60517019, 6.90775528])